In [18]:
import nltk, re, pprint
from nltk import load_parser

In [19]:
nltk.data.show_cfg('grammars/book_grammars/sql0.fcfg')

% start S
S[SEM=(?np + WHERE + ?vp)] -> NP[SEM=?np] VP[SEM=?vp]
VP[SEM=(?v + ?pp)] -> IV[SEM=?v] PP[SEM=?pp]
VP[SEM=(?v + ?ap)] -> IV[SEM=?v] AP[SEM=?ap]
NP[SEM=(?det + ?n)] -> Det[SEM=?det] N[SEM=?n]
PP[SEM=(?p + ?np)] -> P[SEM=?p] NP[SEM=?np]
AP[SEM=?pp] -> A[SEM=?a] PP[SEM=?pp]
NP[SEM='Country="greece"'] -> 'Greece'
NP[SEM='Country="china"'] -> 'China'
Det[SEM='SELECT'] -> 'Which' | 'What'
N[SEM='City FROM city_table'] -> 'cities'
IV[SEM=''] -> 'are'
A[SEM=''] -> 'located'
P[SEM=''] -> 'in'


In [20]:
cp = load_parser('grammars/book_grammars/sql0.fcfg', trace=0)
query = 'What cities are located in China'.split()

In [21]:
#for x in cp.parse(query):
#    print(x)

In [22]:
#for x in cp.parse(query):
#    print(x.draw())

In [23]:
trees = list(cp.parse(query))
answer = trees[0].label()['SEM']
#answer = [x for x in answer if x]
q = ' '.join(answer)
print(q)

SELECT City FROM city_table WHERE   Country="china"


In [24]:
from nltk.sem import chat80
rows = chat80.sql_query('corpora/city_database/city.db', q)
for x in rows:
    print(x[0], end=' ')

canton chungking dairen harbin kowloon mukden peking shanghai sian tientsin 

We need to update SQL grammar to translate the below query:

In [25]:
query2 = 'What cities are in China and have populations above 1,000,000'.split()
query3 = 'What cities have populations above 1,000,000'.split()

In [26]:
# specify and display user grammar

grammar_user = 'sql_grammar.fcfg'
nltk.data.show_cfg(grammar_user)

% start S
S[SEM=(?np + WHERE + ?vp)] -> NP[SEM=?np] VP[SEM=?vp]
VP[SEM=(?v + ?np)] -> TV[SEM=?v] NP[SEM=?np]
VP[SEM=(?v + ?pp)] -> IV[SEM=?v] PP[SEM=?pp]
VP[SEM=(?v + ?ap)] -> IV[SEM=?v] AP[SEM=?ap]
VP[SEM=(?vp1 + ?c + ?vp2)] -> VP[SEM=?vp1] Conj[SEM=?c] VP[SEM=?vp2]
NP[SEM=(?n + ?pp)] -> N[SEM=?n] PP[SEM=?pp]
NP[SEM=(?det + ?n)] -> Det[SEM=?det] N[SEM=?n]
PP[SEM=(?p + ?np)] -> P[SEM=?p] NP[SEM=?np]
AP[SEM=?pp] -> A[SEM=?a] PP[SEM=?pp]
NP[SEM='Country="greece"'] -> 'Greece'
NP[SEM='Country="china"'] -> 'China'
Det[SEM='SELECT'] -> 'Which' | 'What'
N[SEM='City FROM city_table'] -> 'cities'
IV[SEM=''] -> 'are'
A[SEM=''] -> 'located'
P[SEM=''] -> 'in'
TV[SEM=''] -> 'have'
N[SEM='Population'] -> 'populations'
P[SEM='>'] -> 'above'
NP[SEM='1000'] -> '1,000,000'
Conj[SEM='AND'] -> 'and'


In [27]:
cp = load_parser(grammar_user, trace=0)

In [36]:
trees = list(cp.parse(query3))
answer = trees[0].label()['SEM']
#answer = [x for x in answer if x]
q = ' '.join(answer)
print(q)

SELECT City FROM city_table WHERE  Population > 1000


In [29]:
rows = chat80.sql_query('corpora/city_database/city.db', q)
for x in rows:
    print(x[0], end=' ')

athens bangkok barcelona berlin birmingham bombay bucharest budapest buenos_aires cairo calcutta canton chicago chungking delhi detroit glasgow hamburg hongkong_city hyderabad istanbul karachi kyoto leningrad london los_angeles madras madrid manila melbourne mexico_city milan montreal moscow mukden nagoya nanking naples new_york osaka paris peking philadelphia rio_de_janeiro rome santiago sao_paulo seoul shanghai singapore_city sydney tehran tientsin tokyo vienna yokohama 

We now need to accommodate the second clause 'are in China':

In [31]:
# Looking up conjunction implementation

nltk.data.show_cfg('grammars/book_grammars/sql1.fcfg')

% start S
S[SEM=(?np + WHERE + ?vp)] -> NP[SEM=?np] VP[SEM=?vp]
VP[SEM=(?v + ?pp)] -> IV[SEM=?v] PP[SEM=?pp]
VP[SEM=(?v + ?ap)] -> IV[SEM=?v] AP[SEM=?ap]
VP[SEM=(?v + ?np)] -> TV[SEM=?v] NP[SEM=?np]
VP[SEM=(?vp1 + ?c + ?vp2)] -> VP[SEM=?vp1] Conj[SEM=?c] VP[SEM=?vp2]
NP[SEM=(?det + ?n)] -> Det[SEM=?det] N[SEM=?n]
NP[SEM=(?n + ?pp)]  -> N[SEM=?n] PP[SEM=?pp]
NP[SEM=?n]  -> N[SEM=?n]  | CardN[SEM=?n] 
CardN[SEM='1000'] -> '1,000,000' 
PP[SEM=(?p + ?np)] -> P[SEM=?p] NP[SEM=?np]
AP[SEM=?pp] -> A[SEM=?a] PP[SEM=?pp]
NP[SEM='Country="greece"'] -> 'Greece'
NP[SEM='Country="china"'] -> 'China'
Det[SEM='SELECT'] -> 'Which' | 'What'
Conj[SEM='AND'] -> 'and'
N[SEM='City FROM city_table'] -> 'cities'
N[SEM='Population'] -> 'populations'
IV[SEM=''] -> 'are'
TV[SEM=''] -> 'have'
A -> 'located'
P[SEM=''] -> 'in'
P[SEM='>'] -> 'above'


In [35]:
# Running the parser after updating the user grammar

trees = list(cp.parse(query2))
answer = trees[0].label()['SEM']
#answer = [x for x in answer if x]
q = ' '.join(answer)
print(q)

SELECT City FROM city_table WHERE   Country="china" AND  Population > 1000


In [34]:
rows = chat80.sql_query('corpora/city_database/city.db', q)
for x in rows:
    print(x[0], end=' ')

canton chungking mukden peking shanghai tientsin 